# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
from skimage import io
from skimage.transform import rescale, resize, downscale_local_mean

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
#train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
#val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 50 #experiment with the batch size

In [4]:
#pre-processing for augmentation to our images representing video frames
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    vertical_flip=False,
    horizontal_flip=True,
    fill_mode='reflect',
    zoom_range=[0.9, 1.25],
    shear_range=0.01,
    brightness_range=[0.5, 1.5])

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
def generator(source_path, folder_list, batch_size):
    #print( 'Source path = ', source_path, '; batch size =', batch_size)
    #img_idx = list(range(0, 30)) #create a list of image numbers you want to use for a particular video
    img_idx = [1,3,5,7,9,11,13,15,17,19,21,23,25,27,29]
    x = len(img_idx) # number of images for each video
    y = 120          # height of each input image
    z = 120          # width of each input image
    large=(360, 360, 3)
    small=(120, 160, 3)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #print("Reading ", source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    image = io.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    #print("image shape is ", image.shape)
                    #plt.imshow(image)
                    #yield
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if (image.shape == large):
                        image = resize(image, (y, z), anti_aliasing=False)
                    else:
                        image = image[:, 20:140, :]
                    
                    #image augmentation - start
                    if np.random.randn() > 0:
                        image = datagen.random_transform(image)                    
                    #image augmentation - start
                    
                    #print("new image shape is ", image.shape)
                    #plt.imshow(image)
                    #yield
                    
                    #normalize 
                    image = image/255
                    #img_chnl_sum = image[:,:,0] + image[:,:,1] + image[:,:,2]
                    batch_data[folder,idx,:,:,0] = image[:, :, 0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:, :, 1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:, :, 2] #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        
        # write the code for the remaining data points which are left after full batches
        start_idx = num_batches * batch_size
        remaining_size = len(folder_list) - start_idx
        if remaining_size > 0:
            batch_data = np.zeros((remaining_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((remaining_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(start_idx, len(folder_list)): #iterate over the remaining folders
                imgs = os.listdir(source_path+'/'+ t[folder].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #print("Reading ", source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    image = io.imread(source_path+'/'+ t[folder].strip().split(';')[0]+'/'+imgs[item])
                    #print("image shape is ", image.shape)
                    #plt.imshow(image)
                    #yield
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if (image.shape == large):
                        image = resize(image, (y, z), anti_aliasing=False)
                    else:
                        image = image[:, 20:140, :]

                    #image augmentation - start
                    if np.random.randn() > 0:
                        image = datagen.random_transform(image)                    
                    #image augmentation - start
                
                    #print("new image shape is ", image.shape)
                    #plt.imshow(image)
                    #yield
                    #img_chnl_sum = image[:,:,0] + image[:,:,1] + image[:,:,2]
                    batch_data[folder - start_idx,idx,:,:,0] = image[:, :, 0] #normalise and feed in the image
                    batch_data[folder - start_idx,idx,:,:,1] = image[:, :, 1] #normalise and feed in the image
                    batch_data[folder - start_idx,idx,:,:,2] = image[:, :, 2] #normalise and feed in the image

            batch_labels[folder - start_idx, int(t[folder].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
    return None, None

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
#train_path = '/notebooks/storage/Final_data/Collated_training/train'
#val_path = '/notebooks/storage/Final_data/Collated_training/val'
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

#### Next we will try out Conv2D+RNN(GRU)

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, LSTM, GaussianNoise
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.regularizers import l2

initialiser = 'glorot_uniform'

model = Sequential()

model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2), padding='same',
                                 kernel_initializer=initialiser, kernel_regularizer=l2(0.01)),
                          input_shape=(15, 120, 120, 3)))
model.add(GaussianNoise(0.1, input_shape=(15,)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(32, (2, 2), kernel_initializer=initialiser, kernel_regularizer=l2(0.01))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',
                                 kernel_initializer=initialiser, kernel_regularizer=l2(0.01))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
# conv
model.add(TimeDistributed(Conv2D(64, (2, 2), padding='same',
                                 kernel_initializer=initialiser, kernel_regularizer=l2(0.01))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
# max pool
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Flatten()))
#model.add(GRU(64, return_sequences=False, dropout=0.5))
model.add(GRU(64, return_sequences=False))
model.add(Dense(5, activation='softmax'))

In [8]:
optimiser = optimizers.SGD(lr=0.01, clipnorm=1.) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 15, 60, 60, 32)    4736      
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 15, 60, 60, 32)    0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 15, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_3 (TimeDist (None, 15, 60, 60, 32)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 15, 59, 59, 32)    4128      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 15, 59, 59, 32)    128       
_________________________________________________________________
time_distributed_6 (TimeDist (None, 15, 59, 59, 32)    0         
__________

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.000001, verbose=1, cooldown=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [12]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps/batch_size, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


14/14 [==============================] - 84s 6s/step - loss: 3.0162 - categorical_accuracy: 0.1993 - val_loss: 3.4161 - val_categorical_accuracy: 0.1600

Epoch 00001: saving model to model_init_2019-06-2111_08_58.843624/model-00001-3.08941-0.20965-3.41613-0.16000.h5
Epoch 2/10
14/14 [==============================] - 60s 4s/step - loss: 2.9816 - categorical_accuracy: 0.2546 - val_loss: 3.3133 - val_categorical_accuracy: 0.2000

Epoch 00002: saving model to model_init_2019-06-2111_08_58.843624/model-00002-3.05100-0.25189-3.31332-0.20000.h5
Epoch 3/10
14/14 [==============================] - 61s 4s/step - loss: 2.9395 - categorical_accuracy: 0.2567 - val_loss: 3.4005 - val_categorical_accuracy: 0.1800

Epoch 00003: saving model to model_init_2019-06-2111_08_58.843624/model-00003-3.01034-0.26998-3.40055-0.18000.h5
Epoch 4/10
14/14 [==============================] - 60s 4s/step - loss: 2.9128 - categorical_accuracy: 0.2686 - val_loss: 3.3947 - val_categorical_accuracy: 0.1800

Epoch 00004: